In [22]:
import os

In [23]:
%pwd

'/Users/pavankumar/Projects/Text-Summariser-NLP-Project-End-to-end'

In [24]:
os.chdir("/Users/pavankumar/Projects/Text-Summariser-NLP-Project-End-to-end/")
%pwd

'/Users/pavankumar/Projects/Text-Summariser-NLP-Project-End-to-end'

## Entity

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

## Configuration

In [26]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
     def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
          
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

     def get_data_transformation_config(self) -> DataTransformationConfig:
         config = self.config.data_transformation
         create_directories([config.root_dir])

         data_transformation_config = DataTransformationConfig(
              root_dir = config.root_dir,
              data_path = config.data_path,
              tokenizer_name = config.tokenizer_name
          )

         return data_transformation_config


## Components

In [27]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length = 1025, truncation = True)

        with self.tokenizer.as_target_tokenizer():
             target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return{
         'input_ids' : input_encoding['input_ids'],
         'attention_mask' : input_encoding['attention_mask'],
         'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        logger.info("Converting examples to features")

        dataset_samsum = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

        logger.info("Finished converting examples to features")


## Pipeline

In [29]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
     raise e

[2024-04-28 14:51:46,498: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-04-28 14:51:46,500: INFO: common: yaml file:params.yaml loaded successfully]
[2024-04-28 14:51:47,982: INFO: 55964738: Converting examples to features]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/pavankumar/anaconda3/envs/texts/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 299619.31 examples/s]

[2024-04-28 14:51:50,144: INFO: 55964738: Finished converting examples to features]
